In [1]:
import define_transformations_afolu as dta
import define_transformations_circular_economy as dtc
import define_transformations_energy as dte
import define_transformations_ippu as dti

import os, os.path
import numpy as np
import pandas as pd
#from model_attributes import *
import model_attributes as ma
from attribute_table import AttributeTable
import build_composite_baseline_inputs as bcb
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_electricity as ml
import model_energy as me
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import sisepuede_data_api as api
import sisepuede_models as sm
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
from typing import Union
import inspect
import ingestion as ing
import logging
from sisepuede_file_structure import *
import transformations_base_afolu as tba
import transformations_base_circular_economy as tbc
import transformations_base_energy as tbe
import transformations_base_general as tbg
import transformations_base_ippu as tbi


importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(tbc)
importlib.reload(tbe)


import sisepuede as ssp
def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "build_data_file.log"))



# initialize relevant models
model_afolu = mafl.AFOLU(sa.model_attributes)
model_energy = me.NonElectricEnergy(sa.model_attributes)
model_elec = ml.ElectricEnergy(
    sa.model_attributes, 
    sa.dir_jl,
    sa.dir_ref_nemo,
    initialize_julia = False
)

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2632: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2389: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for desig, df in df_by_designation:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2632: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector a

##  BUILD CODE HERE FOR CONVERTING DATA FROM REPO AND OTHER SOURCES

In [2]:
#
# use 062012 for AfOLU
#   replace Sequestration Factors
#   replace soc/climate factors
# pull CE from repo
# IPPU from repo
# 
importlib.reload(api)

# set repository directory for SISEPUEDE Data
dir_repo_data = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data"

# supporting classes
regions = sc.Regions(sa.model_attributes)
time_periods = sc.TimePeriods(sa.model_attributes)

# sisepuede data "api"

repo = api.SISEPUEDEBatchDataRepository(
    dir_repo_data,
    sa.model_attributes
)
# get some subsector specific data


# get particular data from CE, IP, and SE, then select from afolu
modvars = ["Circular Economy", "Energy", "IPPU", "Socioeconomic"]
modvars += [
    model_afolu.modvar_agrc_frac_dry,
    model_afolu.modvar_agrc_frac_no_till,
    model_afolu.modvar_agrc_frac_temperate,
    model_afolu.modvar_agrc_frac_tropical,
    model_afolu.modvar_agrc_frac_wet,
    #model_afolu.modvar_agrc_yf,
    model_afolu.modvar_frst_frac_temperate_nutrient_poor,
    model_afolu.modvar_frst_frac_temperate_nutrient_rich,
    model_afolu.modvar_frst_frac_tropical,
    model_afolu.modvar_lndu_frac_dry,
    model_afolu.modvar_lndu_frac_mineral_soils,
    model_afolu.modvar_lndu_frac_temperate,
    model_afolu.modvar_lndu_frac_tropical,
    model_afolu.modvar_lndu_frac_wet,
    model_afolu.modvar_soil_organic_c_stocks
]

df_sectored_repo_0 = repo.read(modvars, add_time_periods = True)

df_sectored_repo = (
    df_sectored_repo_0
    .dropna()
    .reset_index(drop = True)
)


# get efs from AFOLU corner of repo
"""modvars_lndu_ef = [
    model_afolu.modvar_frst_sq_co2,
    model_afolu.modvar_lndu_ef_co2_conv,
]
df_efs_lndu = repo.read(modvars_lndu_ef, add_time_periods = True)
df_efs_lndu.drop([time_periods.field_year], axis = 1, inplace = True)""";



def modify_pij_to_restrict_transitions(
    df_in: pd.DataFrame,
) -> pd.DataFrame:
    """
    Modify df_in to restrict transition probabilities into primary forest
        (only allowed from other forests)
    """
    attr_lndu = sa.model_attributes.get_attribute_table(sa.model_attributes.subsec_name_lndu)
    model_afolu = mafl.AFOLU(sa.model_attributes)

    cats_skip = [
        model_afolu.cat_lndu_fstm,
        model_afolu.cat_lndu_fstp,
        model_afolu.cat_lndu_fsts
    ]
    
    
    ##  RESTRICT TRANSITIONS INTO PRIMARY FOREST
    
    # get some indices
    inds_to_zero = [
        attr_lndu.get_key_value_index(x) for x in attr_lndu.key_values
        if x not in cats_skip
    ]
    ind_fstp = attr_lndu.get_key_value_index(model_afolu.cat_lndu_fstp)
    
    # setup matrix adjustment points
    dict_adj = dict(
        ((x, ind_fstp), 0) for x in inds_to_zero
    )
    
    
    
    ##  RESTRICT TRANSITIONS OUT OF PRIMARY FOREST
    
    cats_to_zero = [
        model_afolu.cat_lndu_fstm
    ]
    # get some indices
    inds_to_zero = [
        attr_lndu.get_key_value_index(x) for x in attr_lndu.key_values
        if x in cats_to_zero
    ]
    ind_fstp = attr_lndu.get_key_value_index(model_afolu.cat_lndu_fstp)
    
    # setup matrix adjustment points
    dict_adj.update(
        dict(
            ((ind_fstp, x), 0) for x in inds_to_zero
        )
    )
    
    
    
    # check for emission factors, can treat as dummies
    fields_ef = sa.model_attributes.build_varlist(
        None,
        model_afolu.modvar_lndu_ef_co2_conv,
    )
    df_in_copy = df_in.copy()
    for field in fields_ef:
        if field not in df_in_copy.columns:
            df_in_copy[field] = 0.0

    
    # loop by region
    df_out = []
    df_g = df_in_copy.groupby([field_iso])
    
    for iso, df in df_g:
        
        # get matrices
        qs, efs = model_afolu.get_markov_matrices(
            df, 
            len(df)
        )
        # adjust to prevent inflows
        for i, q in enumerate(qs):
            qmat = q
            qs[i] = model_afolu.adjust_transition_matrix(q, dict_adj)
        
        # convert back to df
        df_component_new = model_afolu.format_transition_matrix_as_input_dataframe(qs)
        
        for field in df_component_new.columns:
            df[field] = np.array(df_component_new[field])
    
        df_out.append(df)
        
    df_out = pd.concat(df_out, axis = 0)
        
    return df_out

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1351: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1351: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1351: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped

In [3]:

##########################
#   LOAD INPUT TABLES    #
##########################

warnings.filterwarnings("ignore")
importlib.reload(ml)
importlib.reload(me)


regions = sc.Regions(sa.model_attributes)
time_periods = sc.TimePeriods(sa.model_attributes)

##  setup location of calibrated files by sector


df_fake_data = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
dir_calibs = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo"
fp_export = os.path.join(dir_calibs, "input_all_sectors.csv")



# setup output templates 
dir_template_inputs = os.path.join(dir_calibs, "template_inputs")
os.makedirs(dir_template_inputs, exist_ok = True) if not os.path.exists(dir_template_inputs) else None
dict_template_file_paths = {}


dict_calibration_file_paths = {
    "af": os.path.join(dir_calibs, "af", "data_afolu_partial_2023_06_12.csv"),
    "ce": os.path.join(dir_calibs, "all", "Data_Input_2023_06_30.csv"),
    "en": os.path.join(dir_calibs, "all", "Data_Input_2023_06_30.csv"),
    "ip": os.path.join(dir_calibs, "all", "Data_Input_2023_06_30.csv"),
    # use most recent for socioeconomic
    "se": os.path.join(dir_calibs, "all", "Data_Input_2023_06_30.csv")
    
    #"ce": os.path.join(dir_calibs, "ce", "data_complete_future_2022_12_13_test.csv"),
    #"en": os.path.join(dir_calibs, "en", "data_complete_future_2023_02_24.csv"),#"data_complete_future_2022_12_09_test.csv"),
    #"ip": os.path.join(dir_calibs, "ip", "data_ippu_2023_01_16_fixed_cleaned.csv"),
    # use most recent for socioeconomic
    #"se": os.path.join(dir_calibs, "en", "data_complete_future_2023_06_22.csv")
}

# TEMP FOR INDIA
running_out_of_lac = False
if running_out_of_lac:
    print("READING NON-STANDARD DATA")
    for key in dict_calibration_file_paths.keys():
        dict_calibration_file_paths.update({key: os.path.join(dir_calibs, "all", "Data_Input_2023_05_12_MNE.csv")})

"""
dict_calibration_file_paths = {
    "af": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    "ce": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    "en": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),#"data_complete_future_2022_12_09_test.csv"),
    "ip": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    # use most recent for socioeconomic
    "se": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv")
}
""";


# initialize 
all_regions = None
dict_isos = {}
dict_calibration_tables = {}
dict_replace_iso = sa.model_attributes.dict_attributes.get("region").field_maps.get("iso_alpha_3_to_region")
dict_sets = {}
field_country = "country"
field_iso = "iso_code3"
field_region = "nation"
field_year = "year"
fields_drop = [field_iso, field_region, field_year] # only apply later
field_time_period = sa.model_attributes.dim_time_period

# get some attributes
attr_region = sa.model_attributes.dict_attributes.get("region")
attr_sector = sa.model_attributes.dict_attributes.get("abbreviation_sector")
attr_strat = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_strategy_id}")
attr_time_period = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_time_period}")

# dictionary and attribute derivaties
dict_strat_to_strat_id = attr_strat.field_maps.get("strategy_to_strategy_id")
dict_repl_time_period = attr_time_period.field_maps.get(f"year_to_{sa.model_attributes.dim_time_period}")
strat_base = int(attr_strat.table[attr_strat.table["baseline_strategy_id"] == 1][attr_strat.key])
# map each country to ISO code 3 and each code to 
dict_country_to_iso = dict((k, v.upper()) for k, v in attr_region.field_maps.get(f"{attr_region.key}_to_iso_alpha_3").items())
dict_iso_to_country = sf.reverse_dict(dict_country_to_iso)
all_iso = list(dict_iso_to_country.keys())

# some time period oriented tools
dict_year_to_time_period = attr_time_period.field_maps.get(f"year_to_{attr_time_period.key}")
def year_to_tp(
    year: int
) -> int:

    m = min((dict_year_to_time_period.keys()))
    m_v = dict_year_to_time_period.get(m)

    diff = m - m_v

    return dict_year_to_time_period.get(year, year - diff)





#
#    SOME FUNCTIONS    #
#

# function to verify time periods
def check_time_periods(
    df_in: pd.DataFrame,
    attribute_time_period: AttributeTable,
    fields_group_by: List[str]
) -> pd.DataFrame:
    """
    Check a data frame to ensure that a record exists for each time period
        for the grouping specified in fields_group_by
    """
    
    # check fields
    fields_group_by = [x for x in fields_group_by if x in df_in.columns]
    if len(fields_group_by) == 0:
        return None
    
    field_time_period = attribute_time_period.key
    
    dfs = df_in.groupby(fields_group_by)
    df_out = []
    for df in dfs:
        i, df = df
        
        tps = sorted(list(df[field_time_period]))
        tps_set = sorted(list(set(tps)))
        if (tps == tps_set) & (set(tps) == set(attribute_time_period.key_values)):
            df_out.append(df) 
    
    return pd.concat(df_out, axis = 0).reset_index(drop = True)



def scale_vars(
    df_in: pd.DataFrame,
    dict_scale: Dict[str, float],
    model_attributes: ModelAttributes
) -> pd.DataFrame:
    """
    Scale model variables k in a data frame according to 
        dict_scale = {k: scalar}
    """
    
    df_out = df_in.copy()
    for modvar in dict_scale.keys():
        subsec = model_attributes.get_variable_subsector(modvar)
        fields = model_attributes.build_varlist(
            subsec,
            modvar
        ) if (subsec is not None) else None
        
        if fields is not None:
            df_out[fields] *= dict_scale.get(modvar)
        
    return df_out



def build_yf(
    df_data: pd.DataFrame,
    growth_rate_by_time_period: float,
    time_periods_hist: List[int],
    model_attributes: ma.ModelAttributes,
    add_region: bool = False,
    field_iso: Union[str, None] = None,
    field_region: Union[str, None] = None,
    input_regions_only: bool = False,
    model_afolu: Union[mafl.AFOLU, None] = None,
    modvar: Union[str, None] = None,
    **kwargs,
) -> Union[pd.DataFrame, None]:
    """
    Build yield factors from data frame df_data
    
    Function Arguments
    ------------------
    - df_data: DataFrame containing input data 
    - growth_rate_by_time_period: growth rate to apply to post-historical
        time periods
    - time_periods_hist: time periods containing historical data
    - model_attributes: ModelAttributes object spcifying 
    
    Keyword Arguments
    -----------------
    - add_region: add region field?
    - field_iso: field to use as ISO string
    - field_region: field storing regions/countries
    - input_regions_only: only run for regions associated with input file?
    - model_afolu: optional specification of model_afolu for method and 
        property access
    - modvar: model variable to project
    - **kwargs: passed to bcb.project_from_growth_rate()
    """
    
    return_none = not isinstance(df_data, pd.DataFrame)
    return_none |= not sf.islistlike(time_periods_hist)
    if return_none:
        return None
    
    model_afolu = (
        mafl.AFOLU(model_attributes)
        if not isinstance(model_afolu, mafl.AFOLU)
        else model_afolu
    )
    regions = sc.Regions(model_attributes)
    
    modvar = (
        model_afolu.modvar_agrc_yf
        if modvar is None
        else modvar
    )
    
    # add with time period
    flds = [field_iso]
    flds += (
        [time_periods.field_time_period]
        if (time_periods.field_time_period in df_data.columns)
        else []
    )
    flds += sa.model_attributes.build_varlist(
        None,
        modvar
    )
    
    s1 = set(flds) - set([time_periods.field_time_period])
    if not s1.issubset(set(df_data.columns)):
        return None
    
    missing_regions_process = (
        None
        if input_regions_only
        else "substitute_closest"
    )
    # 
    df_yield_factors = bcb.project_from_growth_rate(
        (
            df_data[
                df_data[time_periods.field_time_period].isin(time_periods_hist)
            ][flds]
            .reset_index(drop = True)
        ), 
        sa.model_attributes,
        modvar,
        growth_rate_by_time_period,
        field_iso = field_iso,
        missing_regions_process = missing_regions_process,
        **kwargs
    )
    
    # 
    if add_region:
        df_yield_factors = regions.add_region_or_iso_field(
            df_yield_factors,
            field_iso = field_iso,
            field_region = field_region,
        )
    
    return df_yield_factors



def fields_to_val(
    df_in: pd.DataFrame,
    dict_modvars: Dict[str, Any],
) -> pd.DataFrame:
    """
    Using dict_modvars, which maps a modvar to some fixed value, map
        modvar fields to value (dictionary value) in df_in
    """
    #
    dict_field_to_val = {}
    for modvar, val in dict_modvars.items():
        
        fields = sa.model_attributes.build_varlist(
            None,
            modvar
        )
        
        dict_field_to_val.update(
            dict((x, val) for x in fields)
        )
        
    
    # update in data frame
    for x, v in dict_field_to_val.items():
        df_in[x] = v
        
    return df_in



# set up the MSP Max Prod growth rate for hydropower--basically, stop it 
def transformation_change_msp_max(
    df_input: pd.DataFrame,
    dict_cat_to_vector: Dict[str, float],
    model_attributes: ma.ModelAttributes,
    model_electricity: ml.ElectricEnergy,
    drop_flag: Union[int, float, None] = None,
    model_energy: Union[me.NonElectricEnergy, None] = None,
    **kwargs
) -> pd.DataFrame:
    """ 
    Modify the maximum growth in production by electricity generation technology
        in serve of MinShareProduction specifications. Used to prevent a
        technology's estimated production from growing (or to place a cap on 
        growth in production) after the last zero-valued time-period
        associated with `vec_ramp`. 
    
    For exampmle, can be used to prevent hydropoower prodction fom increasing 
        after a given time period (based on `vec_ramp`).

    Function Arguments
    ------------------
    - df_input: input data frame containing baseline trajectories
    - dict_cat_to_vector: dictionary mapping a technology category to two an 
        input vector. The vector uses a drop flag to (generally -999) to identify
        time periods that are not subject to an MSP Max Prod; other values greater
        than 0 are used to identify the maximum deviation from the *last time 
        period with a non-drop flag*, entered as a proportion.
    - model_attributes: ModelAttributes object used to call strategies/variables
    - model_electricity: Electricity and Fuel Production model used to call 
        variables
    - vec_ramp: ramp vec used for implementation

    Keyword Arguments
    -----------------
    - drop_flag: value in 
        model_electricity.modvar_entc_max_elec_prod_increase_for_msp used to 
        signal the presence of no constraint. Defaults to 
        model_electricity.drop_flag_tech_capacities if None
    - field_region: field in df_input that specifies the region
    - regions_apply: optional set of regions to use to define strategy. If None,
        applies to all regions.
    - strategy_id: optional specification of strategy id to add to output
        dataframe (only added if integer)
    """
    
    drop_flag = model_electricity.drop_flag_tech_capacities if not sf.isnumber(drop_flag) else drop_flag
    
    # check for variables and initialize fields_check as drops
    df_out = df_input.copy()
    fields_check = model_attributes.build_varlist(None, model_electricity.modvar_entc_max_elec_prod_increase_for_msp)
    df_out[fields_check] = drop_flag
    
    for cat, vec in dict_cat_to_vector.items():
        
        dict_trans = {
            model_elec.modvar_entc_max_elec_prod_increase_for_msp: {
                "bounds": (drop_flag, np.inf),
                "categories": [cat],
                "magnitude": vec,
                "magnitude_type": "vector_specification",
                "vec_ramp": vec
            }
        }
        
        # call general transformation
        df_out = tbe.transformation_general(
            df_out,
            model_attributes,
            dict_trans,
            model_energy = model_electricity.model_energy,
            **kwargs
        )

        
    return df_out





########################
###                  ###
###    BEGIN TEMP    ###
###                  ###
########################

###
###    TEMPORARY: OVERWRITE CAPACITIES WITH EARLIER DATA
###

df_residual_capacity_overwrite = pd.read_csv(sa.fp_csv_nemomod_residual_capacity_inputs)
df_residual_capacity_overwrite = df_residual_capacity_overwrite[
    df_residual_capacity_overwrite["year"].isin(dict_repl_time_period.keys())
]

dict_replace_iso_rev = sf.reverse_dict(dict_replace_iso)
dict_replace_iso_rev = dict((k, v.upper()) for k, v in dict_replace_iso_rev.items())
df_residual_capacity_overwrite["year"].replace(dict_repl_time_period, inplace = True)

df_residual_capacity_overwrite.drop([attr_region.key], axis = 1, inplace = True)
df_residual_capacity_overwrite.rename(columns = {
    "year": sa.model_attributes.dim_time_period,
    "iso_alpha_3": field_iso
}, inplace = True)


###
###  TEMPORARY: OVERWRITE AFOLU CLIMATE DATA AND SOC
###

# KCC
df_kcc = pd.read_csv(sa.fp_csv_climate_fields_by_country_simple)
df_kcc.rename(
    columns = {regions.field_iso: field_iso}, 
    inplace = True
)
#df_kcc[field_iso] = df_kcc[field_country].replace(dict_country_to_iso)
#df_kcc.drop([field_country], axis = 1, inplace = True)

# SOC
df_soc = pd.read_csv(sa.fp_csv_soc_fields_by_country_simple)
df_soc.rename(
    columns = {regions.field_iso: field_iso}, 
    inplace = True
)

#df_soc[field_iso] = df_soc[field_country].replace(dict_country_to_iso)
#df_soc.drop([field_country], axis = 1, inplace = True)

df_climate = pd.merge(df_kcc, df_soc, how = "inner")




###
###  TEMPORARY: OVERWRITE SOME MORE ENERGY DATA AND ADD IN DATA FROM REPO
###

"""
fp_data_overwrite = os.path.join(dir_calibs, "all", "Data_Input_2023_06_30.csv")
df_overwrite = pd.read_csv(fp_data_overwrite)
# fill in Infs
df_overwrite.replace([np.inf, -np.inf], 0.0, inplace = True)


fields_drop = [x for x in df_overwrite.columns if ("unnamed" in x.lower())]
fields_drop += [x for x in df_overwrite.columns if ("gas_furnace" in x) or ("gas_petroleum_liquid" in x)]
fields_drop += [x for x in df_overwrite.columns if ("residual_capacity" in x)]
fields_drop += [x for x in ["nemomod_entc_total_annual_max_capacity_pp_hydropower_gw"] if x in df_overwrite.columns]
fields_drop_ow = [x for x in df_overwrite.columns if ("variable_cost" in x) and ("per_gw" in x)]

df_overwrite = time_periods.years_to_tps(
    df_overwrite,
    field_year = "Year"
)
df_overwrite[sa.model_attributes.dim_time_period] = df_overwrite["Year"].apply(year_to_tp)
df_overwrite.drop(fields_drop, axis = 1, inplace = True) if (len(fields_drop) > 0) else None
""";

###
###    get AFOLU probabilties from september 2022  (overwrites df_sectored_repo, so merge it in after)
###

df_afolu_for_probs = pd.read_csv(
    os.path.join(dir_calibs, "af", "data_afolu_partial_202209.csv")
)
fields = sa.model_attributes.build_varlist(
    None,
    model_afolu.modvar_lndu_prob_transition
)
df_afolu_for_probs = df_afolu_for_probs[[field_iso, time_periods.field_time_period] + fields];
# ensure that the probabilities into forest primary are 0 except for forests
df_afolu_for_probs = modify_pij_to_restrict_transitions(df_afolu_for_probs)

# OVE
df_overwrite = df_afolu_for_probs.copy()
"""
sf.match_df_to_target_df(
    df_overwrite,
    df_sectored_repo,
    fields_index = [sa.model_attributes.dim_time_period, field_iso],
    overwrite_only = False
)
df_overwrite = sf.match_df_to_target_df(
    df_overwrite,
    df_afolu_for_probs,
    fields_index = [sa.model_attributes.dim_time_period, field_iso],
    overwrite_only = False
)#pd.merge(df_overwrite, df_afolu_for_probs, on = [time_periods.field_time_period, field_iso])

# merge/interpolate to deal with any missing years
df_all_rows = sf.explode_merge(
    attr_time_period.table[[attr_time_period.key]].drop_duplicates(),
    df_overwrite[[field_iso]].drop_duplicates()
)
df_overwrite = pd.merge(
    df_all_rows, 
    df_overwrite, 
    how = "left"
).interpolate().drop(["Year", "Nation"], axis = 1).sort_values(by = [field_iso, attr_time_period.key]).reset_index(drop = True)
"""


###
###    TEMPORARY: OVERWRITE INITIAL PRODUCTION
###

df_prodvalues = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ippu/ProductionValues_2023_02_17.csv")
field_tp = sa.model_attributes.dim_time_period
df_tp = sa.model_attributes.dict_attributes.get(f"dim_{field_tp}").table[[field_tp]]
df_prodvalues = sf.explode_merge(df_prodvalues, df_tp) if (field_tp not in df_prodvalues.columns) else df_prodvalues
df_prodvalues["nation"].replace(dict_replace_iso_rev, inplace = True)
df_prodvalues.rename(
    columns = {
        "nation": field_iso
    }, 
    inplace = True
)


###
###    TEMPORARY: OVERWRITE SELECT EFFICIENCY FACTORS REPRESENTING IMPROVEMENTS IN ELECTRIFYING HEAT ENERGY TO 
###

fields_overwrite = sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_enfu,
    "Average Industrial Energy Fuel Efficiency Factor",
    restrict_to_category_values = ["fuel_electricity"]
) + sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_scoe,
    "SCOE Efficiency Factor for Heat Energy from Electricity"
)
df_effic_factors = df_fake_data[[field_time_period] + fields_overwrite]



###
###    TEMPORARY: OVERWRITE SEQUESTRATION FACTORS IN FORESTS
###

df_overwrite_sequestration_factors = sa.model_attributes.get_standard_variables(
    df_fake_data,
    "Forest Sequestration Emission Factor"
)
df_overwrite_sequestration_factors = pd.concat(
    [
        df_fake_data[[attr_time_period.key]].copy(), 
        df_overwrite_sequestration_factors
    ],
    axis = 1
)



###
###    TEMPORARY: CAP MAX TECH CAPACITY FOR HYDROPOWER
###

df_hydro_cap_synthetic = pd.read_csv(sa.fp_csv_nemomod_hydropower_max_tech_capacity)



###
###    TEMPORARY: READ IN MinShareProduction BASELINE VALUES
###


df_min_share_production = pd.read_csv(
    sa.fp_csv_nemomod_minimum_share_of_production_baselines
).drop(["year"], axis = 1)

if False:
    # temporary modification - try cutting the MSP in hal for the first 6 years (while capping new investments)
    df_min_share_production.loc[
        [x < 6 for x in list(df_min_share_production[sa.model_attributes.dim_time_period])], 
        [x for x in df_min_share_production.columns if "nemomod" in x]
    ] = df_min_share_production.loc[
        [x < 6 for x in list(df_min_share_production[sa.model_attributes.dim_time_period])], 
        [x for x in df_min_share_production.columns if "nemomod" in x]
    ] * 0.0
    
def rescale_msps(
    df_msp: pd.DataFrame,
    field_time_period: str = sa.model_attributes.dim_time_period,
    min_total: Union[float, None] = 1.0,
    time_periods_apply: Union[List[int], None] = None
) -> pd.DataFrame:
    """
    Rescale minimum shares of production to meet a minimum total
    
    Function Arguments
    ------------------
    - df_msp: data frame containing variables associatedion with
        model_elec.modvar_entc_nemomod_min_share_production that are 
        to be rescaled (potentially) to meet a minimum total
    
    Keyword Arguments
    -----------------
    - field_time_period: field containing the time period
    - min_total: minimum total for minimum shares of production
    - time_periods_apply: time periods to apply minimum to 
    """
    
    if min_total is None:
        return df_msp
    
    # get time periods that are to be rescaled
    time_periods_apply = (
        list(df_msp[field_time_period].unique()) 
        if (time_periods_apply is None)
        else time_periods_apply
    )
    
    
    # variables to rescale
    vars_rescale = sa.model_attributes.build_varlist(
        None,
        model_elec.modvar_entc_nemomod_min_share_production
    )
    vars_rescale = [x for x in df_msp.columns if x in vars_rescale]

    # split off appropriate years
    df_rescale = df_msp[df_msp[field_time_period].isin(time_periods_apply)].copy().reset_index(drop = True)
    df_ignore = df_msp[~df_msp[field_time_period].isin(time_periods_apply)].copy().reset_index(drop = True)
    
    # get totals and finally rescale
    vec_total = np.array(df_rescale[vars_rescale].sum(axis = 1))
    vec_min = min_total*np.ones(len(df_rescale))
    vec_scale = sf.vec_bounds(vec_min/vec_total, (1, np.inf))
    df_rescale[vars_rescale] = sf.do_array_mult(np.array(df_rescale[vars_rescale]), vec_scale)
    
    
    df_msp = (
        pd.concat([df_rescale, df_ignore], axis = 0)
        .sort_values(by = [field_iso, field_time_period])
        .reset_index(drop = True)
    )
    
    
    return df_msp, vars_rescale

# RESCALE MSP AT EARLY TIME PERIODS TO BE MORE SIGNFICANT (TEMPORARY SOLUTION 04-21-23--REAL SOLUTION SHOULD INCLUDE BETTER RESIDUAL CAPACITY/CAPACITY FACTORS)
df_min_share_production, vars_rescale = rescale_msps(df_min_share_production, time_periods_apply = list(range(6)))
    


###
###    TEMPORARY: READ IN TRANSMISSION LOSSES BY COUNTRY
###

df_transmission_losses = pd.read_csv(
    sa.fp_csv_nemomod_transmission_losses
).drop(["year"], axis = 1)
df_transmission_losses = df_transmission_losses[
    df_transmission_losses[attr_time_period.key].isin(attr_time_period.key_values)
].reset_index(drop = True)



###
###    TEMPORARY: READ IN FUEL COSTS
###

df_fuel_costs = pd.read_csv(sa.fp_csv_nemomod_fuel_costs)



###
###    TEMPORARY: GET TECHNOLOGY MAX CAPACITY INVESTMENTS
###

fields_overwrite = sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_entc,
    model_elec.modvar_entc_nemomod_total_annual_max_capacity_investment
)
df_max_cap = df_fake_data[[field_time_period] + fields_overwrite]



###
###  TEMPORARY OVERWRITE OF WOOD FUEL FRACTIONS FROM RECYCLED WOOD 
###


modvars_frac = model_energy.modvars_inen_list_fuel_fraction
dict_cp_wood = {}

# set cateogories
attr_ind = sa.model_attributes.dict_attributes.get("cat_industry")
dict_ind_to_recycling_target = attr_ind.field_maps.get("cat_industry_to_target_cat_industry_to_adjust_with_recycling")

# get target category (wood) and source from recycling
cat_target = model_afolu.cat_ippu_wood
cat_source = None
for k in dict_ind_to_recycling_target.keys():
    cat_source = k if (dict_ind_to_recycling_target.get(k) == f"``{cat_target}``") else cat_source


if cat_source is not None:
    for modvar in modvars_frac:

        cats = sa.model_attributes.get_variable_categories(modvar)
        subsec = sa.model_attributes.get_variable_subsector(modvar)
        
        if (cat_source in cats) and (cat_target in cats):

            field_source = sa.model_attributes.build_varlist(
                subsec,
                modvar,
                restrict_to_category_values = [cat_source]
            )[0]

            field_target = sa.model_attributes.build_varlist(
                subsec,
                modvar,
                restrict_to_category_values = [cat_target]
            )[0]

            dict_cp_wood.update({field_source: field_target})

            
###
###   TEMP: DEFINE FUNCTION TO ADD BIOFUELS (BIOMASS) TO MANUFACTURING INDUSTRIES (excl electronics, cement, metals, glass, lime_and_carbonites)
###
###   04-21-2023
###

cats_inen_with_biofuels = [
    "chemicals",
    "other_industries",
    "other_product_manufacturing",
    "paper",
    "plastic",
    "textiles",
    "wood"
]


def modify_input_inen_fracs(
    df_energy_trajectories_in: pd.DataFrame,
    dict_fuel_info: Dict,
    cats_to_apply: Union[List[str], None] = None,
    field_time_period: str = sa.model_attributes.dim_time_period,
    regions: Union[List, None] = None
) -> pd.DataFrame:
    """
    Rescale minimum shares of production to meet a minimum total.
    
    Assumes targets in dict_fuel_info have total of 0.
    
    
    Function Arguments
    ------------------
    - df_energy_trajectories: data frame containing INEN fractions
        to be modified
    - dict_fuel_info: dictionary mapping fuel to two keys:
        * "target": target fraction to apply to all time periods
        * "cats": catgories to apply to
    - regions: regions to apply to
    """
    # some initialization
    df_energy_trajectories = df_energy_trajectories_in.copy()
    attr_enfu = sa.model_attributes.dict_attributes.get("cat_fuel")
    attr_ind = sa.model_attributes.dict_attributes.get("cat_industry")
    cats = attr_ind.key_values if (cats_to_apply is None) else cats_to_apply
    
    # get target total
    targ_total = [v for k, v in dict_fuel_info.items() if k in attr_enfu.key_values]
    targ_total = float(sf.vec_bounds(sum(targ_total), (0, 1)))
    scalar_existing = 1 - targ_total
    
    dict_inen_fracs = model_energy.dict_inen_fuel_categories_to_fuel_variables
    regions = (
        sorted(list(df_energy_trajectories[field_iso].unique()))
        if regions is None
        else regions
    )
    df_grouped_by_region = df_energy_trajectories.groupby([field_iso])
    
    df_out = []
    
    for region, df in df_grouped_by_region:
        
        if region in regions:
            for fuel in dict_inen_fracs.keys():
                
                dict_vars_cur = dict_inen_fracs.get(fuel)    
                dict_fuel_info_cur = dict_fuel_info.get(fuel)
                
                modvar = dict_vars_cur.get("fuel_fraction")
                
                # get categories, check against spcificationin Model Attributes, then convert to fields
                cats_fields = sa.model_attributes.get_variable_categories(modvar)
                cats_fields = (
                    [x for x in cats if x in cats_fields]
                    if cats_fields is not None
                    else None
                )
                fields = sa.model_attributes.build_varlist(
                    None,
                    modvar,
                    restrict_to_category_values = cats_fields
                )
                
                # only adjust fields that are present
                fields_adj = [x for x in fields if x in df.columns]
                df[fields_adj] = (
                    np.array(df[fields_adj])*scalar_existing
                    if fuel not in dict_fuel_info.keys()
                    else dict_fuel_info.get(fuel)
                )
            
        df_out.append(df)
        
    df_out = (
        pd.concat(df_out, axis = 0)
        .sort_values(by = [field_iso, field_time_period])
        .reset_index(drop = True)
    )
        
      
    return df_out



###
###    TEMP: ADD IN SCOE ELASTS -- MOVE TO BUILD TRANSFORMATIONS FILE FUNCTION/PROCESS
###


def get_afolu_data(
) -> pd.DataFrame:
    """
    get afolu data
    """
    fps_read = [
        # co2 factors from LU conversion
        sa.fp_csv_lndu_ef_conversion_co2,
        # forest sequestration factors
        sa.fp_csv_lndu_ef_forest_sequestration_co2,
        # Import fractions and export totals
        sa.fp_csv_afolu_import_export_estimates,
    ]
    
    df_out = None
    
    
    for fp in fps_read:
        
        df_tmp = pd.read_csv(fp)
        
        (
            df_tmp.rename(columns = {regions.field_iso: repo.field_repo_iso}, inplace = True)
            if regions.field_iso in df_tmp.columns
            else None
        )
        
        df_out = (
            df_tmp
            if df_out is None
            else pd.merge(df_out, df_tmp, how = "inner")
        )
    
    fields_ind = [time_periods.field_time_period, repo.field_repo_iso]
    fields_dat = sorted([x for x in df_out.columns if x not in fields_ind])
    df_out = df_out[fields_ind + fields_dat]
    
    return df_out



def get_energy_data(
) -> pd.DataFrame:
    fps_read = [
        # Elasticity of SCOE energy consumption to GDP/Capita
        sa.fp_csv_scoe_elasticity_of_energy_consumption,
        # SCOE Energy Consumption Scalar
        sa.fp_csv_scoe_consumption_scalar,
        # Initial SCOE Energy consumption
        sa.fp_csv_scoe_initial_energy_consumption,
    ]
    
    df_out = None
    
    
    for fp in fps_read:
        
        df_tmp = pd.read_csv(fp)
        
        df_out = (
            df_tmp
            if df_out is None
            else pd.merge(df_out, df_tmp, how = "inner")
        )

    df_out = time_periods.years_to_tps(
        df_out,
        field_year = field_year
    )

    
    return df_out



def get_ippu_data(
) -> pd.DataFrame:
    fps_read = [
        # Elasticity of industrial production to GDP
        sa.fp_csv_elasticity_of_industrial_production,
        # Flourinated Compound emission factors
        sa.fp_csv_ippu_fc_efs,
        # Fraction of cement from clinker
        sa.fp_csv_ippu_frac_cement_clinker,
        # Industrial Production Scalar
        sa.fp_csv_industrial_production_scalar,
        # Initial Industrial production
        sa.fp_csv_initial_industrial_production,
        # Net Imports clinker
        sa.fp_csv_ippu_net_imports_cement_clinker,
        
    ]
    
    df_out = None
    
    
    for fp in fps_read:
        
        df_tmp = pd.read_csv(fp)
        
        df_out = (
            df_tmp
            if df_out is None
            else pd.merge(df_out, df_tmp, how = "inner")
        )

    df_out = time_periods.years_to_tps(
        df_out,
        field_year = field_year
    )

    
    return df_out


df_ener_overwrites = get_energy_data()
df_ippu_overwrites = get_ippu_data()
df_afolu_overwrites = get_afolu_data()




######################
###                ###
###    END TEMP    ###
###                ###
######################
dict_fields_missing_all = {}
if True:
    # load in tables and make some quick modifications
    for k in dict_calibration_file_paths.keys():
        
        # read and clean columns--base one sectored repo and overwrite with data specified in df_read
        df_read = pd.read_csv(dict_calibration_file_paths.get(k))
        dict_rnm = dict((x, x.lower()) for x in df_read.columns if x != x.lower())
        df_read.rename(columns = dict_rnm, inplace = True)
        
        # drop unused indices
        fields_drop = [x for x in df_read.columns if "unnamed" in str(x).lower()]
        (
            df_read.drop(fields_drop, axis = 1, inplace = True) 
            if (len(fields_drop) > 0) 
            else None
        )

        # drop ISOs that are undefiuned
        df_read = df_read[df_read[field_iso].isin(all_iso)].reset_index(drop = True)
        # add time period if not present
        if attr_time_period.key not in df_read.columns:
            df_read[attr_time_period.key] = df_read[field_year].apply(time_periods.year_to_tp)
        
        df_read = sf.match_df_to_target_df(
            df_sectored_repo,
            df_read,
            fields_index = [sa.model_attributes.dim_time_period, field_iso],
            overwrite_only = False
        )
        
        

        ###
        ###  TEMPORARY: DROP OLD FUELS
        ###
        if True:
            fields_drop = [x for x in df_read.columns if ("gas_furnace" in x) or ("gas_petroleum_liquid" in x)]
            (
                df_read.drop(fields_drop, axis = 1, inplace = True) 
                if (len(fields_drop) > 0) 
                else None
            )
        
        
        ###
        ###  TEMPORARY: OVERWRITE AFOLU CLIMATE DATA
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_climate,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        
        
        ###
        ###    TEMPORARY: OVERWRITE OTHER KEY ENERGY DATA AND SECTORED DATA
        ###
        if True and not running_out_of_lac:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_overwrite,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
        
           
        ###
        ###    TEMPORARY: OVERWRITE FUEL COSTS
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_fuel_costs,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
            
        
        ###
        ###    TEMPORARY: OVERWRITE MAX CAPACITY
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_max_cap,
                fields_index = [sa.model_attributes.dim_time_period],
                overwrite_only = False
            )
            
            
        
            
            
        ###
        ###    TEMPORARY: RENAME FROM CALIBRATED FILE
        ###
        
        if False:
            dict_repl = {
                "gas_petroleum_liquid": "hydrocarbon_gas_liquids",
                "gas_furnace": "furnace_gas"
            }

            dict_rnm = {}

            for substr in dict_repl.keys():
                dict_rnm_cur = dict(
                    (x, x.replace(substr, dict_repl.get(substr))) 
                    for x in df_read.columns if substr in x
                )

                dict_rnm.update(dict_rnm_cur)

            df_read.rename(columns = dict_rnm, inplace = True)

        
        ####
        ####   TEMPORARY OVERWRITE OF RESIDUAL CAPACITY
        ####

        print("TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED")
        
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_residual_capacity_overwrite,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
        #print(list(df_read.columns)[0:4])
        ####
        ####   TEMPORARY OVERWRITE OF ELECTRICAL TRANSMISSION LOSSES
        ####

        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_transmission_losses,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ####
        ####   TEMPORARY OVERWRITE OF INITIAL PRODUCTION
        ####
        if False:
            # DEPRECATED AS OF 26042023--INTEGRATED INTO RBD PIPELINE
            df_read = sf.match_df_to_target_df(
                df_read,
                df_prodvalues,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: OVERWRITE EFFICIENCY FACTORS FOR RENEWABLES IN ENTC
        ###
        
        if True:
            fields = sa.model_attributes.build_varlist(
                "Energy Technology",
                "Technology Efficiency of Fuel Use",
                restrict_to_category_values = ["pp_hydropower", "pp_geothermal", "pp_solar", "pp_wind", "pp_ocean"]
            )
            df_read[fields] = 1
        

        ###
        ###    TEMPORARY: OVERWRITE EFFICIENCY FACTORS FOR ELECTRICITY HEAT IN SCOE/INEN
        ###
        
        if True:
            df_effic_factors_merge = sf.explode_merge(
                df_effic_factors,
                df_read[[field_iso]].drop_duplicates()
            )
            df_read = sf.match_df_to_target_df(
                df_read,
                df_effic_factors_merge,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: ADD HYDROPOWER GENERATION CONSTRIANT
        ###

        if False:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_hydro_cap_synthetic,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: OVERWRITE MinShareProduction for Baseline
        ###

        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_min_share_production,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            #print(df_read[df_read[field_iso].isin(["BRA"])][[x for x in df_read.columns if "min_share_prod" in x]].head())
      
        
        

        # filter time periods and do some field cleaning
        if field_time_period not in df_read.columns:
            df_read[field_time_period] = np.array(np.array(df_read[field_year]) - 2015).astype(int)
            df_read = df_read[
                df_read[field_time_period].isin(attr_time_period.key_values)
            ].reset_index(drop = True)
        df_read = df_read[df_read[field_time_period] >= 0].reset_index(drop = True)
        df_read[field_region] = [dict_replace_iso.get(x.lower()) for x in list(df_read[field_iso])]

        # fields missing from input file: take from fake data
        fields_missing = list(set(df_fake_data.columns) - set(df_read.columns))
        fields_eliminate = list((set(df_read.columns) - set(df_fake_data.columns)) - set(fields_drop) - set(["strategy_id"]))
        
        dict_fields_missing_all.update({k: fields_missing})
        df_read = pd.merge(
            df_read, 
            df_fake_data[[field_time_period] + fields_missing],
            on = [field_time_period],
            how = "left"
        )
        
        
        ###
        ###    TEMPORARY: OVERWRITE VALUES WITH FIXED VALUES SET IN DICTIONARY
        ###
        
        if True:
            dict_fields_to_value = dict(
                (x, 1.0) 
                for x in df_read.columns 
                if x.startswith("demscalar")
                and x not in (list(df_ippu_overwrites.columns) + list(df_ener_overwrites.columns))
            )
            
            for fld in dict_fields_to_value.keys():
                val = dict_fields_to_value.get(fld)
                df_read[fld] = val
                
            print("NOTE: OVERWRITE DEMSCALARS AS 1; SHOULD BE INSTITUTED IN DATA FILE")
                
                
        
        ###
        ###  TEMPORARY OVERWRITE OF WOOD FUEL FRACTIONS FROM RECYCLED WOOD 
        ###

        if False:
            for field_old in dict_cp_wood.keys():
                field_new = dict_cp_wood.get(field_old)
                df_read[field_new] = list(df_read[field_old])

            
            
            
        ###
        ###  TEMPORARY: OVERWRITE KEY IPPU AND CLINKER DATA
        ###
        
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_ippu_overwrites,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
            
        
        ###
        ###  TEMPORARY: OVERWRITE KEY ENERGY CONSUMPTION DATA
        ###
        
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_ener_overwrites,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        
        
        
        ###
        ###  TEMPORARY: OVERWRITE FOREST SEQUESTRATION
        ###
        
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_overwrite_sequestration_factors,
                fields_index = [sa.model_attributes.dim_time_period],
                overwrite_only = False
            )
            
            print("Using FAKE DATA sequestration (20230612) in absence of calibration. Check after calibration")
            
            
            
        ###
        ###    TEMPORARY: FIX CATTLE POPS
        ###
        
        # TEMPORARY FIX: WARN EXCESSIVELY

        flds_halve = ["pop_lvst_initial_cattle_dairy", "pop_lvst_initial_cattle_nondairy"]
        for fld in flds_halve:
            if fld in df_read.columns:
                df_read[fld] = np.round(np.array(df_read[fld])/2, decimals = 0).astype(int)
        print(f"WARNING! modifying fields {flds_halve} (found cattle pops were doubled on 20230614) to cut in half.\nVERIFY WHETHER THIS HAS BEEN FIXED AT EACH ITERATION\n. E.g., per https://ourworldindata.org/grapher/cattle-livestock-count-heads, pop of Brazil in 2021 should be around 220 million")
        
        dfr = df_read.copy()
        
        
        
        ###
        ###    TEMPORARY: SET FRAC EATING RED MEAT + LVST CARRYING CAPACITY SCALAR TO 1
        ###
        
        if True:
            df_read = fields_to_val(
                df_read,
                {
                    model_afolu.model_socioeconomic.modvar_gnrl_frac_eating_red_meat: 1.0,
                    model_afolu.modvar_lvst_carrying_capacity_scalar: 1.0,
                }
            )
            
            
            
            
        ###
        ###    TEMPORARY: BUILD YIELD FACTORS AND ADD IN LAND USE EMISSION FACTORS (AF-SPECIFIC )
        ###
        
        if k == "af":
            # specify yield factor as 1.6% growth
            df_yf = (
                build_yf(
                    df_read, 
                    0.016,
                    list(range(5)),
                    sa.model_attributes,
                    add_region = False,
                    field_iso = field_iso,
                )
            )
            
            # modify lvst carrying capacity scalar - set to constant
            if True:
                """
                df_lvst_carrying_capacity = (
                    build_yf(
                        df_read, 
                        0.007037, # should lead to 20% by 2050; 1.2^(1/25)
                        list(range(9)),
                        sa.model_attributes,
                        add_region = False,
                        field_iso = field_iso,
                        modvar = model_afolu.modvar_lvst_carrying_capacity_scalar,
                    )
                )
                df_yf = pd.merge(df_yf, df_lvst_carrying_capacity)
                """
                
                varname = sa.model_attributes.build_varlist(
                    None,
                    model_afolu.modvar_lvst_carrying_capacity_scalar,
                )
                for field in varname:
                    df_read[field] = 1
                
            # specify livestock exports
            df_lvst_exports = (
                build_yf(
                    df_read, 
                    0.02,
                    list(range(6)),
                    sa.model_attributes,
                    add_region = False,
                    field_iso = field_iso,
                    modvar = model_afolu.modvar_lvst_equivalent_exports,
                )
            )
            df_yf = pd.merge(df_yf, df_lvst_exports)
            

            df_read = sf.match_df_to_target_df(
                df_read,
                df_yf,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
            
            df_read = sf.match_df_to_target_df(
                df_read,
                df_afolu_overwrites,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )

        
        
        # filter out regions with invalid time period specifications
        df_read = check_time_periods(
            df_read,
            attr_time_period,
            [field_iso]
        )   
        # update available iso codes
        dict_isos.update({k: set(df_read[field_iso])})
        
        # fix region issue
        df_read = (
            pd.merge(
                df_read.drop([field_region], axis = 1),
                attr_region.table[[attr_region.key, "iso_alpha_3"]].rename(
                    columns = {
                        "iso_alpha_3": field_iso,
                        attr_region.key: field_region
                    }
                )
            )
            
        )

        dict_calibration_tables.update({k: df_read})
        set_merge = set(df_read[field_region])
        dict_sets.update({k: set_merge})
        all_regions = set_merge if (all_regions is None) else (all_regions & set_merge)
        
        
        # write file for templatization input
        fp_write = os.path.join(dir_template_inputs, f"template_input_{k}.csv")
        dict_template_file_paths.update({k: fp_write})
        
        print(f"Writing sector '{k}' to {fp_write}...")
        dict_calibration_tables.get(k).to_csv(
            fp_write,
            index = None,
            encoding = "UTF-8"
        )
        print("Done.\n\n")

    attr_region = sa.model_attributes.dict_attributes.get("region")
    all_regions = sorted(list(set(attr_region.key_values) & all_regions))








TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
NOTE: OVERWRITE DEMSCALARS AS 1; SHOULD BE INSTITUTED IN DATA FILE
Using FAKE DATA sequestration (20230612) in absence of calibration. Check after calibration
WARNING! modifying fields ['pop_lvst_initial_cattle_dairy', 'pop_lvst_initial_cattle_nondairy'] (found cattle pops were doubled on 20230614) to cut in half.
VERIFY WHETHER THIS HAS BEEN FIXED AT EACH ITERATION
. E.g., per https://ourworldindata.org/grapher/cattle-livestock-count-heads, pop of Brazil in 2021 should be around 220 million
Writing sector 'af' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_af.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
NOTE: OVERWRITE DEMSCALARS AS 1; SHOULD BE INSTITUTED IN DATA FILE
Using FAKE DATA sequestration (20230612) in absence of calibration. Check after calibration
WARNING! modifying fields ['pop_lv

In [4]:
df_complete = None
dfs_join = []

fields_ind = [
    time_periods.field_time_period,
    field_iso,
    field_region
]

attr_sector = sa.model_attributes.dict_attributes.get("abbreviation_sector")
dict_abv_to_sector = attr_sector.field_maps.get(f"{attr_sector.key}_to_sector")

for k, v in dict_calibration_tables.items():

    sector = dict_abv_to_sector.get(k)
    if (sector is None) | (not isinstance(v, pd.DataFrame)):
        continue
    
    # build extraction fields for this sector
    fields_keep = fields_ind.copy()
        
    subsectors = sa.model_attributes.get_sector_subsectors(sector)
    for subsec in subsectors:
        fields_keep += sa.model_attributes.build_varlist(subsec)
    
    fields_keep = [x for x in fields_keep if x in v.columns]
    
    
    df_complete = (
        v[fields_keep]
        if df_complete is None
        else pd.merge(df_complete, v[fields_keep])
    )


all_fields_req = list(sa.model_attributes.build_variable_dataframe_by_sector(None, include_time_periods = False)["variable"])
all_fields_missing = [x for x in all_fields_req if x not in df_complete]

if True & (len(all_fields_missing) == 0):
    df_complete.to_csv(
        fp_export,
        index = None,
        encoding = "UTF-8"
    )
    print(f"successfully exported all inputs to {fp_export}")
else:
    warnings.warn(f"MISSING FILES {all_fields_missing}")

In [8]:
df_complete.to_csv(
    fp_export,
    index = None,
    encoding = "UTF-8"
)

# Derivative inputs
- put together any inputs here that might depend on inputs from the complete file (e.g., exports are based on historical fractions)

In [578]:
df_sectored_repo_for_imp_exp = repo.read(["AFOLU", "Socioeconomic"], add_time_periods = True)
# rely on adjusted pop from above and drop repo-based yield factors
df_sectored_repo_for_imp_exp.drop(
    [
        x for x in df_sectored_repo_for_imp_exp.columns 
        if ("pop_lvst_init" in x)
        or x.startswith("yf_agrc_")
    ], 
    axis = 1, 
    inplace = True
)
df_sectored_repo_for_imp_exp.dropna(inplace = True)
#

df_inputs_for_aie = sf.match_df_to_target_df(
    df_complete,
    df_sectored_repo_for_imp_exp,
    fields_index = [sa.model_attributes.dim_time_period, repo.field_repo_iso],
    overwrite_only = False
)




def get_afolu_imp_exps(
    df_input: pd.DataFrame,
    export_growth_rate: float = 0.015,
    restrict_imp_exp_years: Union[List, None] = list(range(2010, 2019)),
) -> pd.DataFrame:
    
    field_nation = "Nation"
    
    
    df_afolu_imp_exp = pd.read_csv(sa.fp_csv_afolu_import_exports)
    
    # clean names
    dict_repl = {
        "bolivia_(plurinational_state_of)": "bolivia",
        "china,_hong_kong_sar": "hong_kong",
        "china,_macao_sar": "macao",
        "congo": "republic_of_the_congo",
        "democratic_people's_republic_of_korea": "north_korea",
        "iran_(islamic_republic_of)": "iran",
        "lao_peoples_democratic_republic": "lao",
        "micronesia_(federated_states_of)": "micronesia",
        "russian_federation": "russia",
        "venezuela_(bolivarian_republic_of)": "venezuela",
        "united_kingdom_of_great_britain_and_northern_ireland": "united_kingdom",
    }
    vec = [
        str(x).lower().replace(" ", "_").replace("'", "").replace("-", "_").replace("ô", "o")
        for x in df_afolu_imp_exp[field_nation]
    ]
    vec = [dict_repl.get(x, x) for x in vec]
    df_afolu_imp_exp[field_nation] = vec
    
    # some continued cleaning
    df_afolu_imp_exp = regions.add_region_or_iso_field(
        df_afolu_imp_exp, 
        field_region = field_nation
    )
    df_afolu_imp_exp = time_periods.years_to_tps(
        df_afolu_imp_exp, 
    )
    
    # filter out some years
    all_years = sorted(list(df_afolu_imp_exp[time_periods.field_year]))
    restrict_years = (
        all_years
        if not sf.islistlike(restrict_imp_exp_years)
        else [x for x in all_years if x in restrict_imp_exp_years]
    )
    if len(restrict_years) == 0:
        return None
    
    df_afolu_imp_exp = (
        df_afolu_imp_exp[
            df_afolu_imp_exp[time_periods.field_year].isin(restrict_years)
        ]
        .reset_index(drop = True)
        .rename(
            columns = {
                regions.field_iso: repo.field_repo_iso
            }
        )
        .drop([field_nation], axis = 1)
    )
    
    
    # loop over shared regions to get lvst data (crops to come)
    
    df_out = []
    flds_ind = [
        repo.field_repo_iso, 
        time_periods.field_time_period, 
        time_periods.field_year
    ]
    
    regions_loop = sorted(list(set(df_afolu_imp_exp[repo.field_repo_iso]) & set(df_input[repo.field_repo_iso])))
    
    for iso in regions_loop:

        df_af = (
            df_afolu_imp_exp[df_afolu_imp_exp[repo.field_repo_iso].isin([iso])]
            .sort_values(by = [time_periods.field_time_period])
            .reset_index(drop = True)
            .interpolate(method = "bfill")
            .interpolate(method = "ffill")
            .interpolate()
            .copy()
        )
        df_in = (
            df_input[df_input[repo.field_repo_iso].isin([iso])]
            .sort_values(by = [time_periods.field_time_period])
            .reset_index(drop = True)
            .drop(
                [x for x in df_input.columns if (x in df_af.columns) & (x not in flds_ind)],
                axis = 1
            )
            .copy()
        )

        df_both = pd.merge(
            df_af, 
            df_in,
            how = "inner"
        )
        
        # project out yields and populations
        try:
            df_afolu_project = model_afolu(df_both)
            print(f"iso {iso} succeeded")
        except:
            continue
            
      
    
        ##  DO AGRC CALCS
        
        global arr_agrc_yields
        arr_agrc_yields = sa.model_attributes.get_standard_variables(
            df_afolu_project,
            model_afolu.modvar_agrc_yield,
            expand_to_all_cats = True,
            return_type = "array_base",
            var_bounds = (0, np.inf)
        )
        
        arr_agrc_frac_exported = sa.model_attributes.get_standard_variables(
            df_both,
            model_afolu.modvar_agrc_equivalent_exports,
            expand_to_all_cats = True,
            return_type = "array_base",
            var_bounds = (0, 1.0)
        )
    
        # get exports for ag
        df_agrc_exports = arr_agrc_yields*arr_agrc_frac_exported
        df_agrc_exports = model_afolu.model_attributes.array_to_df(
            df_agrc_exports,
            model_afolu.modvar_agrc_equivalent_exports,
            reduce_from_all_cats_to_specified_cats = True,
        )
        df_agrc_exports[time_periods.field_time_period] = list(df_both[time_periods.field_time_period])
        df_agrc_exports[repo.field_repo_iso] = iso
        
        # build exports df
        df_agrc_exports = build_yf(
            df_agrc_exports, 
            export_growth_rate,
            list(df_agrc_exports[time_periods.field_time_period]),
            model_afolu.model_attributes,
            add_region = False,
            field_iso = repo.field_repo_iso,
            input_regions_only = True,
            modvar = model_afolu.modvar_agrc_equivalent_exports,
        )
        df_out_iso = df_agrc_exports.copy()
        
        
        # get import fractions
        df_agrc_frac_imports = sa.model_attributes.get_standard_variables(
            df_both,
            model_afolu.modvar_agrc_frac_demand_imported,
            expand_to_all_cats = False,
            return_type = "data_frame",
            var_bounds = (0, 1.0)
        ).copy()
        
        df_agrc_frac_imports[time_periods.field_time_period] = list(df_both[time_periods.field_time_period])
        df_agrc_frac_imports[repo.field_repo_iso] = iso
        
        # build exports df
        df_agrc_frac_imports = build_yf(
            df_agrc_frac_imports, 
            0.0,
            list(df_agrc_frac_imports[time_periods.field_time_period]),
            model_afolu.model_attributes,
            add_region = False,
            field_iso = repo.field_repo_iso,
            input_regions_only = True,
            modvar = model_afolu.modvar_agrc_frac_demand_imported,
        )

        df_out_iso = pd.merge(df_out_iso, df_agrc_frac_imports)
        
        
        
        ##  DO LVST CALCULATIONS
        
        # get population and export fraction
        arr_lvst_pop = sa.model_attributes.get_standard_variables(
            df_afolu_project,
            model_afolu.modvar_lvst_pop,
            expand_to_all_cats = True,
            return_type = "array_base",
            var_bounds = (0, np.inf)
        )

        arr_lvst_export_frac = sa.model_attributes.get_standard_variables(
            df_both,
            model_afolu.modvar_lvst_equivalent_exports,
            expand_to_all_cats = True,
            return_type = "array_base",
            var_bounds = (0, 1)
        )
        
        # get exports for lvst
        df_lvst_exports = arr_lvst_pop*arr_lvst_export_frac
        df_lvst_exports = model_afolu.model_attributes.array_to_df(
            df_lvst_exports,
            model_afolu.modvar_lvst_equivalent_exports,
            reduce_from_all_cats_to_specified_cats = True,
        )
        df_lvst_exports[time_periods.field_time_period] = list(df_both[time_periods.field_time_period])
        df_lvst_exports[repo.field_repo_iso] = iso
        
        # build exports df
        df_lvst_exports = build_yf(
            df_lvst_exports, 
            export_growth_rate,
            list(df_lvst_exports[time_periods.field_time_period]),
            model_afolu.model_attributes,
            add_region = False,
            field_iso = repo.field_repo_iso,
            input_regions_only = True,
            modvar = model_afolu.modvar_lvst_equivalent_exports,
        )
        
        df_out_iso = pd.merge(df_out_iso, df_lvst_exports)

        
        
        ##  add in LVST import fractions
        
        # get import fractions
        df_lvst_frac_imports = sa.model_attributes.get_standard_variables(
            df_both,
            model_afolu.modvar_lvst_frac_demand_imported,
            expand_to_all_cats = False,
            return_type = "data_frame",
            var_bounds = (0, 1.0)
        ).copy()
        
        df_lvst_frac_imports[time_periods.field_time_period] = list(df_both[time_periods.field_time_period])
        df_lvst_frac_imports[repo.field_repo_iso] = iso
        
        # build exports df
        df_lvst_frac_imports = build_yf(
            df_lvst_frac_imports, 
            0.0,
            list(df_lvst_frac_imports[time_periods.field_time_period]),
            model_afolu.model_attributes,
            add_region = False,
            field_iso = repo.field_repo_iso,
            input_regions_only = True,
            modvar = model_afolu.modvar_lvst_frac_demand_imported,
        )
        
        df_out_iso = pd.merge(df_out_iso, df_lvst_frac_imports)
        
        df_out.append(df_out_iso)
    
    
    df_out = pd.concat(df_out, axis = 0).reset_index(drop = True)
    df_out.fillna(0.0, inplace = True)
    
    return df_out

In [579]:
# for this one, use AFOLU and SOCIOECONOMIC inputs 




#df = df_compllete#dict_calibration_tables.get("af")
df_afolu_impo_exps = get_afolu_imp_exps(df_inputs_for_aie)#[df[repo.field_repo_iso].isin(["BRA"])])

df_afolu_impo_exps.to_csv(
    sa.fp_csv_afolu_import_export_estimates,
    index = None,
    encoding = "UTF-8"
)

iso ARG succeeded
iso BOL succeeded
iso BRA succeeded
iso CHL succeeded
iso COL succeeded
iso CRI succeeded
iso DOM succeeded
iso ECU succeeded
iso GTM succeeded
iso HND succeeded
iso JAM succeeded
iso MEX succeeded
iso NIC succeeded
iso PAN succeeded
iso PER succeeded
iso PRY succeeded
iso SLV succeeded
iso URY succeeded


# SOME INTERMEDIATE (TEMPORARY) FIXES

##  Correct landfill, open dumping, and incineration fractions to sum to 1 in waste (20230627)

In [211]:
model_circecon = mc.CircularEconomy(sa.model_attributes)
modvars = model_circecon.modvars_waso_frac_non_recyled_pathways

df_ce_vars = repo.read(modvars, add_time_periods = True)
fields = [
    sa.model_attributes.build_varlist(None, x)[0]
    for x in modvars
]

# normalize the array
arr = np.array(df_ce_vars[fields])
arr = sf.do_array_mult(arr, 1/arr.sum(axis = 1))
df_ce_vars[fields] = arr

# sort and then write
df_ce_vars = (
    df_ce_vars
    .drop([time_periods.field_time_period], axis = 1)
    .sort_values(by = [repo.field_repo_iso, time_periods.field_year])
    .reset_index(drop = True)
)


repo.write_from_df(
    df_ce_vars,
    list(
        range(
            min(df_ce_vars[time_periods.field_year]), 
            2021
        )
    )
)






##  Disallow transition probabilities into forests_primary EXCEPT for those from secondary forest/mangroves (VERY quick/crude fix)

In [234]:
model_afolu.cat_lndu_fstm

'forests_mangroves'

In [ ]:
tba.transformation_support_lndu_transition_to_category_targets_single_region(
    df_tmp,
    dict_magnitude,
    vec_ramp,
    model_attributes,
    cats_stable = cats_stable,
    max_change_allocated_to_pasture_frac_adjustment = 0.0,
    model_afolu = model_afolu,
    pasture_key = key_pastures,
    **kwargs
)

In [245]:
fld = "pij_lndu_croplands_to_croplands"
df_temp[["iso_code3", "time_period", fld]].sort_values(by = [fld], ascending = False)




,iso_code3,time_period,pij_lndu_croplands_to_croplands
827,SUR,35,1.000000
359,DOM,35,1.000000
863,TTO,35,1.000000
358,DOM,34,0.999999
357,DOM,33,0.999998
...,...,...,...
720,PRY,0,0.955930
252,COL,0,0.955930
74,BRB,2,0.955305
73,BRB,1,0.954546
